In [ ]:
# This code shows example function calls and walks through Sync Module Code. It loads in the current parameter files
# and the module code. Code here will walk through the "main" function and call the helper functions in "sync.jl"

include("../modules/generate_raw_data.jl")
include("../modules/process_raw_data.jl")
include("../modules/geometry.jl")
include("../modules/scene.jl")
#include("inputs/input_parameters_LLH_nadirlooking.jl")
#include("inputs/input_parameters_LLH_slantlooking.jl")
include("../inputs/input_parameters_SCH_lookangle.jl")
include("../modules/range_spread_function.jl") # as RSF
include("../modules/orbits.jl")
include("../modules/sync.jl")
include("../modules/error_sources.jl")

using NCDatasets
using Statistics
using Plots

## PLATFORM LOCATIONS
orbit_filename="../inputs/orbitOutput_082020.nc" # position in km, time in sec
# orbit_dataset=Dataset("inputs/"*orbit_filename) # Read orbits data in NetCDF format
orbit_dataset=Dataset(orbit_filename) # Read orbits data in NetCDF format

t12_orbits=orbit_dataset["time"][1:2] # first two time samples
dt_orbits=t12_orbits[2]-t12_orbits[1] # time resolution of orbits (s)
orbit_time_index=(Int(round(SAR_start_time/dt_orbits))+1:1:Int(round((SAR_start_time+SAR_duration)/dt_orbits))+1) # index range for orbit times for time interval of interest
orbit_time=orbit_dataset["time"][orbit_time_index] # read in time data
orbit_pos=orbit_dataset["position"][:,:,orbit_time_index] # read in position data #TODO convert ECI to ECEF?
slow_time=(SAR_start_time:1/fp:SAR_start_time+SAR_duration) # create slow time axis
orbit_pos_interp=Orbits.interp_orbit(orbit_time,orbit_pos,slow_time) # interpolate orbit to slow time
p_xyz=1e3*orbit_pos_interp # convert km to m
Np=size(orbit_pos)[2] # number of platforms
Nst=size(slow_time)[1] # number of slow-time samples (pulses processed)
# test plot of orbits
plot(slow_time, orbit_pos_interp[:,1,:]', xaxis=("time (sec)"), ylabel=("ECEF position (km)"))


In [ ]:
## TARGET/SCENE LOCATIONS
targets,Nt=Scene.construct_targets_str(target_pos_mode,t_loc_1,t_loc_2,t_loc_3,t_ref) # Nt: number of targets, targets: structure array containing target locations and reflectivities
targets_loc=zeros(3,Nt);for i=1:Nt;targets_loc[:,i]=targets[i].loc;end # 3xN
s_loc_3xN=Scene.form3Dgrid_for(s_loc_1,s_loc_2,s_loc_3) # using 3 nested for loops
t_xyz_3xN,s_xyz_3xN=Scene.convert_target_scene_coord_to_XYZ(ts_coord_sys,s_loc_3xN,targets_loc,p_xyz,look_angle,p_avg_heading,earth_radius,earth_eccentricity)
## TARGET REFLECTIVITIES
targets_ref=zeros(1,Nt);for i=1:Nt;targets_ref[i]=targets[i].ref;end

## RANGE SPREAD FUNCTION (matched filter output)
min_range,max_range=Geometry.find_min_max_range(t_xyz_3xN,p_xyz)
Trx=2*(max_range-min_range)/c+2*pulse_length # s duration of RX window
if enable_fast_time # matched filter gain is included in Srx
    Srx,MF,ft,t_rx=RSF.ideal_RSF(pulse_length,Δt,bandwidth,Trx) # Srx: RX window with MF centered, MF: ideal matched filter output (range spread function, RSF) for LFM pulse, ft: fast-time axis for MF, t_rx: RX window
    # Srx,MF,ft,t_rx=RSF.non_ideal_RSF(pulse_length,Δt,bandwidth,Trx,SFR,window_type) # TODO non-ideal RSF for LFM pulse with system complex frequency response (SFR) and fast-time windowing
end


In [ ]:
## GENERATE RAW DATA
ref_range=Geometry.distance(mean(t_xyz_3xN,dims=2),mean(mean(p_xyz,dims=2),dims=3)) # reference range (equal to slant_range in sch?)
if enable_fast_time # with fastime and slowtime; matched filter gain is included in Srx
    rawdata=Generate_Raw_Data.main_RSF_slowtime(t_xyz_3xN,p_xyz,mode,tx_el,fc,Srx,t_rx,ref_range,targets_ref) # rawdata is a: 3D array of size Nst x Np x Nft (SAR/SIMO), 4D array of size Nst x Np(RX) x Np(TX) x Nft (MIMO)
else # without fastime, with slowtime; matched filter gain is included inside the function
    rawdata=Generate_Raw_Data.main_noRSF_slowtime(t_xyz_3xN,p_xyz,mode,tx_el,fc,targets_ref) # rawdata is a: 2D array of size Nst x Np (SAR/SIMO), 3D array of size Nst x Np(RX) x Np(TX) (MIMO)
end
if !enable_fast_time;SNR=SNR*pulse_length*bandwidth;end # SNR increases after matched filter
if enable_thermal_noise;rawdata=Error_Sources.random_noise(rawdata,SNR,enable_fast_time,mode);end # adding random noise based on SNR after range (fast-time) processing


In [ ]:
#here, instead of calling the error_sources sync function, we'll walk through the sync module functions
# rawdata_sync = Error_Sources.synchronization_errors(rawdata,slow_time,orbit_pos_interp,enable_fast_time,parameters)
include("../inputs/input_parameters_sync.jl")
using Interpolations
using LinearAlgebra
using FFTW
using Plots
using Distributed

(phase_err, sync_PSDs) = Sync.get_sync_phase(slow_time, orbit_pos_interp, parameters);

# phase_err are the output values that gets added to the raw data in the error_sources.jl module
plot(slow_time,phase_err', xlabel = "time (s)", ylabel = "Phase (rad)",title = "RF Phase Errors")

In [ ]:
# Now we can reuse the sync_PSDs that were previously calculated. For long SRI, this is a time saver
include("../modules/sync.jl")
(phase_err2, sync_PSDs) = Sync.get_sync_phase(slow_time, orbit_pos_interp, parameters, sync_PSDs);

# Note: because this is a new instantiation of the phase errors, the values are different! 
# But, the underlying PSD is the same
plot(slow_time,phase_err2', xlabel = "time (s)", ylabel = "Phase (rad)",title = "RF Phase Errors from Reused PSDs")

